In [12]:
!pip install boto3 fabric

In [85]:
# Project settings

PROJECT_NAME = "OpenAssistants"

DOMAIN_NAME = "openassistants.io"

ENVIRONMENTS = {
    'dev': {
        'repo': {
            'url': "https://github.com/rekognize/open-assistants-private",
            'branch': "dev",
        },
        'domain': f"test.{DOMAIN_NAME}",
    },
    'prod': {
        'source': ("https://github.com/rekognize/open-assistants-private", "main"),
        'domain': DOMAIN_NAME,
    },
    'public': {
        'source': ("https://github.com/rekognize/open-assistants", "main"),
        'domain': f"public.{DOMAIN_NAME}",
    }
}


# Instance settings

SECURITY_GROUP_NAME = f"{PROJECT_NAME}-sg"
AMI_ID = "ami-0866a3c8686eaeeba" # Ubuntu Server 24.04 LTS
INSTANCE_TYPE = "t3.micro"
EBS_VOLUME_SIZE = 16  # GB
REGION = "us-east-1"
EC2_USER = "ubuntu"



In [38]:
# overrides for simplicity

DOMAIN_NAME = "test.openassistants.io"


In [97]:
import os
import boto3
from botocore.exceptions import ClientError
from dotenv import load_dotenv

load_dotenv()

s3_client = boto3.client('s3', region_name=REGION)

ec2_client = boto3.client('ec2', region_name=REGION)
ec2_resource = boto3.resource('ec2', region_name=REGION)

ENVIRONMENT = 'dev'

In [ ]:
# Create the S3 bucket

BUCKET_NAME = f'{PROJECT_NAME}-{ENVIRONMENT}'.lower()

s3_client.create_bucket(Bucket=BUCKET_NAME)

In [ ]:
# Create the EC2 instance

import os

# Create the key pair

key_name = f"{PROJECT_NAME}-key"
key_path = os.path.expanduser(f"~/.ssh/{key_name}.pem")

try:
    key_pair = ec2_client.create_key_pair(KeyName=key_name)
except ClientError as e:
    if e.response['Error']['Code'] == "InvalidKeyPair.Duplicate":
        print(f"{key_name} already exists.")
    else:
        raise
else:
    with open(key_path, "w") as file:
        file.write(key_pair["KeyMaterial"])
    os.chmod(key_path, 0o400)


# Create and authorize the security group

try:
    response = ec2_client.create_security_group(
        GroupName=SECURITY_GROUP_NAME,
        Description="Security group for my project",
    )
except ClientError as e:
    if e.response['Error']['Code'] == "InvalidGroup.Duplicate":
        response = ec2_client.describe_security_groups(GroupNames=[SECURITY_GROUP_NAME])
        security_group_id = response['SecurityGroups'][0]['GroupId']
    else:
        raise e
else:
    security_group_id = response["GroupId"]
    ec2_client.authorize_security_group_ingress(
        GroupId=security_group_id,
        IpPermissions=[
            {'IpProtocol': 'tcp', 'FromPort': 22, 'ToPort': 22,
             'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
            {'IpProtocol': 'tcp', 'FromPort': 80, 'ToPort': 80,
             'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
            {'IpProtocol': 'tcp', 'FromPort': 443, 'ToPort': 443,
             'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
        ]
    )

print("Launching EC2 instance...")
response = ec2_client.run_instances(
    ImageId=AMI_ID,
    InstanceType=INSTANCE_TYPE,
    KeyName=key_name,
    SecurityGroupIds=[security_group_id],
    MinCount=1,
    MaxCount=1,
    BlockDeviceMappings=[
        {
            "DeviceName": "/dev/sda1",
            "Ebs": {
                "VolumeSize": EBS_VOLUME_SIZE,
                "DeleteOnTermination": True,
                "VolumeType": "gp3"  # General Purpose SSD (gp3)
            },
        },
    ],
    TagSpecifications=[
        {
            "ResourceType": "instance",
            "Tags": [
                {
                    "Key": "Name",
                    "Value": PROJECT_NAME
                }
            ]
        }
    ]
)

# Get the instance ID of the newly created instance
instance_id = response["Instances"][0]["InstanceId"]
print("EC2 instance created with Instance ID:", instance_id)

# Wait for the instance to reach running state and retrieve the public IP and DNS
instance = ec2_resource.Instance(instance_id)

print("Waiting for the instance to enter 'running' state...")
instance.wait_until_running()
instance.reload()  # Refresh instance attributes

# Get the public IP and DNS for SSH
print("Instance is ready for SSH access.")
print("Public IP:", instance.public_ip_address)
print("Public DNS:", instance.public_dns_name)
print("Use the following command to SSH into the instance:")
print(f"ssh -i {key_path} ubuntu@{instance.public_ip_address}")

In [132]:
# Setup the EC2 instance with Fabric
from fabric import Connection

c = Connection(
    host=instance.public_ip_address, 
    user="ubuntu", 
    connect_kwargs={"key_filename": key_path},
)


In [ ]:
c.run("sudo apt update")
c.run("sudo apt upgrade -y")
c.run("sudo apt install -y python3-pip python3-venv nginx git supervisor")

In [ ]:
# Create and install the Github deploy key

import requests
from urllib.parse import urlparse

# Generate the deploy key with the repo name

deploy_key_path = f'/home/ubuntu/.ssh/{ENVIRONMENTS[ENVIRONMENT]['repo']['url'].split('/')[-1]}_deploy_key'
c.run(f'ssh-keygen -t rsa -b 2048 -f {deploy_key_path} -N ""')


# Install the key
github_token = os.getenv('GITHUB_TOKEN')  # GitHub token with repo access
public_key = c.run(f"cat {deploy_key_path}.pub").stdout.strip()

parsed_url = urlparse(ENVIRONMENTS[ENVIRONMENT]['repo']['url'])
repo_path = parsed_url.path.strip("/")
url = f"https://api.github.com/repos/{repo_path}/keys"

response = requests.post(
    url, 
    headers={
        "Authorization": f"token {github_token}",
        "Accept": "application/vnd.github.v3+json"
    }, 
    json={
        "title": f"{PROJECT_NAME} Deploy Key",
        "key": public_key,
        "read_only": True
    }
)


In [ ]:
# Server configuration

ENVIRONMENT_NAME = "dev"

project_dir = f"/home/ubuntu/{PROJECT_NAME}-{ENVIRONMENT_NAME}"

# SSH URL for GitHub, using SSH key authentication
c.run(f"GIT_SSH_COMMAND='ssh -i {deploy_key_path} -o StrictHostKeyChecking=no' git clone -b {ENVIRONMENTS[ENVIRONMENT_NAME]['repo']['branch']} git@github.com:{repo_path}.git {project_dir}")

c.run(f"cd {project_dir} && python3 -m venv venv && source venv/bin/activate && pip install -r requirements.txt")


In [ ]:
# Copy the .env file for the environment
c.put(f".env-{ENVIRONMENT_NAME}", os.path.join(f"{project_dir}", ".env"))

In [ ]:
# Django migration
c.run(f"cd {project_dir} && source venv/bin/activate && python manage.py migrate")

In [ ]:
# Collect static
# !!! re-run after setting STATIC_ROOT setting
c.run(f"cd {project_dir} && source venv/bin/activate && python manage.py collectstatic --noinput")

In [ ]:
# Create superuser
superuser_name = f"{PROJECT_NAME}-admin"
superuser_email = f"admin@{DOMAIN_NAME}"
superuser_password = f"{PROJECT_NAME}-321*"

# Command to create superuser non-interactively (createsuperuser does not have non-interactive mode)
create_superuser_script = f"""
from django.contrib.auth import get_user_model

User = get_user_model()
if not User.objects.filter(username='{superuser_name}').exists():
    User.objects.create_superuser('{superuser_name}', '{superuser_email}', '{superuser_password}')
"""

# Run the Django shell command to create the superuser
c.run(f"cd {project_dir} && source venv/bin/activate && echo \"{create_superuser_script}\" | python manage.py shell")
